In [1]:
import pandas as pd
import numpy as np
import statistics as stat

In [2]:
data = pd.read_csv('scdb-throughput-40-60-metrics-3.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
#grouping every transaction by its ID
grouped_data = data.groupby('Tx_id')

In [4]:
#return_accept_id_group = data.groupby('Accept_id')

In [5]:
tx_id = list(grouped_data.groups.keys())
len(tx_id)

296

In [6]:
(max(data['time']) - min(data['time']))

758.6434237957001

In [7]:
throughput = len(tx_id) / (max(data['time']) - min(data['time']))
throughput

0.3901701256685667

In [8]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])
rfq_count = 0
bid_count = 0
create_count = 0
accept_count = 0
return_count_1 = 0
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'REQUEST_FOR_QUOTE':
        rfq_count += 1
    if data_unique_ids.iloc[i]['Tx_type'] == 'CREATE':
        create_count += 1    
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_count += 1   
    if data_unique_ids.iloc[i]['Tx_type'] == 'ACCEPT':
        accept_count += 1 
    if data_unique_ids.iloc[i]['Tx_type'] == 'RETURN':
        return_count_1 += 1   
print(f"#RFQs {rfq_count}")
print(f"#BIDs =  {bid_count}")
print(f"#CREATEs =  {create_count}")
print(f"#ACCEPTs =  {accept_count}")
print(f"#RETURNs =  {return_count_1}")
print(f"#Total =  {return_count_1 + rfq_count + bid_count + create_count + accept_count}")

#RFQs 5
#BIDs =  90
#CREATEs =  115
#ACCEPTs =  5
#RETURNs =  81
#Total =  296


In [9]:
# Time per each transaction type
time_all_committed_bids = {}
time_passed_bids = []
bid_time = {}
bids_overfit = []
bids_underfit = []
other_bids = []

time_all_committed_creates = {}
time_passed_creates = []
create_time = {}
creates_overfit = []
creates_underfit = []
other_creates = []

time_all_committed_rfqs = {}
time_passed_rfqs = []
rfq_time = {}
rfq_overfit = []
rfq_underfit = []
other_rfqs = []

time_all_committed_accepts = {}
time_passed_accepts = []
accept_time = {}
time_failed_accepts = []
accept_underfit = []
accept_overfit = []
other_accepts = []


time_all_committed_returns = {}
time_passed_returns= []
return_time = {}
return_overfit = []
return_underfit = []
other_returns = []


time_bids = []
time_id_bid = {}


time_rfqs = []
time_id_rfqs = {}

time_accept = []
time_id_accept = {}

time_create = []
time_id_create = {}

time_returns = []
accept_return_count = {}


time_id_return = {}
for i in range(0, len(tx_id)):
    temp_list = []
    return_count = 0
#     if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list)
#         time_rfqs.append(actual_time)
#         time_id_rfqs[tx_id[i]] = actual_time

    if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            rfq_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(tx_id[i])
            rfq_underfit.append(temp_list)
        elif len(temp_list) <= 10:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            rfq_overfit.append(temp_list)
            
        else:
            temp_list.append(tx_id[i])
            other_rfqs.append(temp_list)
        
        
#     if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list) 
#         time_create.append(actual_time)
#         time_id_create[tx_id[i]] = actual_time

    if "CREATE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list)
            time_all_committed_creates[tx_id[i]] = actual_time
            time_passed_creates.append(actual_time)
            create_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(tx_id[i])
            creates_underfit.append(temp_list)
        elif len(temp_list) <= 10:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_passed_creates.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            creates_overfit.append(temp_list)
            
        else:
            temp_list.append(tx_id[i])
            other_creates.append(temp_list)
    """ 
        if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0] or 'None' != grouped_data.get_group(tx_id[i])["Accept_id"].to_list()[0]:
        print(i)
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] = "RETURN" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data['Time'][j])
        print(temp_list)
        actual_time = max(temp_list)
        time_accept.append(actual_time)
        """
    
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data["Time"][j])
                return_count += 1
        accept_return_count [tx_id[i]] = return_count
        last_return = max(temp_list)
        time_accept.append(last_return)
        time_id_accept[tx_id[i]] = last_return
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            temp_list.append(last_return)
            actual_time = max(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            accept_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(last_return)
            temp_list.append(tx_id[i])
            accept_underfit.append(temp_list)
        elif len(temp_list) <= 10:
            time_all_committed_accepts[tx_id[i]] = actual_time
            temp_list.append(last_return)
            actual_time = max(temp_list)
            time_passed_accepts.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            accept_overfit.append(temp_list)
            
        else:
            temp_list.append(last_return)
            temp_list.append(tx_id[i])
            other_accepts.append(temp_list)

    
#     if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         for j in range(0, len(data)):
#             if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
#                 temp_list.append(data["Time"][j])
#                 return_count += 1
#         accept_return_count [tx_id[i]] = return_count
#         actual_time = max(temp_list)
#         time_accept.append(actual_time)
#         time_id_accept[tx_id[i]] = actual_time
         
    if "RETURN" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) >= 4:
            time_all_committed_returns[tx_id[i]] = temp_list
            actual_time = max(temp_list)
            time_passed_returns.append(actual_time)
            return_time[tx_id[i]] = actual_time
        elif len(temp_list) < 4:
            temp_list.append(tx_id[i])
            return_underfit.append(temp_list)
        elif len(temp_list) > 7:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_passed_returns.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            return_overfit.append(temp_list)
            
        else:
            temp_list.append(tx_id[i])
            other_returns.append(temp_list)
    
#     if "RETURN" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list)
#         time_returns.append(actual_time)
#         time_id_return[tx_id[i]] = actual_time

            
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            time_all_committed_bids[tx_id[i]] = temp_list
            actual_time = max(temp_list)
            time_passed_bids.append(actual_time)
            bid_time[tx_id[i]] = actual_time
        elif len(temp_list) < 6:
            temp_list.append(tx_id[i])
            bids_underfit.append(temp_list)
        elif len(temp_list) == 7:
            temp_list = grouped_data.get_group(tx_id[i]).drop_duplicates(subset=['Event'])["Time"].to_list()
            actual_time = max(temp_list)
            time_passed_bids.append(actual_time)
            temp_list.append(tx_id[i])
            temp_list.append(actual_time)
            bids_overfit.append(temp_list)
            
        else:
            other_bids.append(temp_list)
#     if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
#         temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
#         actual_time = max(temp_list)
#         if actual_time != 0:
#             time_bids.append(actual_time)
#             time_id_bid[tx_id[i]] = actual_time

In [10]:
len(time_id_accept)

5

In [11]:
accept_return_count

{'36610ec0ccd514fcb4e5d1b249fdf0ed90ddf059dbad6c73dbe601bad9b17fb5': 12,
 '464dd71326d80995e46ff6af7e8222509e49ea6ee4e7592348e61cbf24959a59': 15,
 '839c8f24f89666792d22afb28f4845ec71aed61c46ca856a56ac6f2e6d979f35': 18,
 'a9cb288fedbd039c104b59b15d0d7b37a3ddfc0d556aa1d502b39e5ae65ed8a5': 18,
 'bdb61c61793c108026b1bc0d4d6dc27b4c85cc6ab6c820687f81fb2c7c85c960': 18}

In [12]:
# Analysis of CREATE Transaction
time_passed_creates.remove(max(time_passed_creates)) 
create_mean = "{:.2f}".format(stat.mean(time_passed_creates))
create_median = stat.median(time_passed_creates)
create_max = max(time_passed_creates)
create_min = min(time_passed_creates)
create_std_dev = "{:.2f}".format(stat.stdev(time_passed_creates))

print(create_mean)
print(create_median)
print(create_max)
print(create_min)
print(create_std_dev)
print(f"total creates = {len(time_passed_creates)}")

create_dict = {
               'min': create_min,
               'max': create_max,
               'average': create_mean,
               'median' : create_median,
               'standard_deviation' : create_std_dev,
               '# of txs' : create_count}

85.80
85.5
145
67
17.52
total creates = 114


In [13]:
# Analysis of RFQ Transaction
if len(time_passed_rfqs) < 2:
    time_rfqs.append(time_passed_rfqs[0])
rfq_mean = "{:.2f}".format(stat.mean(time_passed_rfqs))
rfq_median = stat.median(time_passed_rfqs)
rfq_max = max(time_passed_rfqs)
rfq_min = min(time_passed_rfqs)
rfq_std_dev = "{:.2f}".format(stat.mean(time_passed_rfqs))

print(rfq_mean)
print(rfq_median)
print(rfq_max)
print(rfq_min)
print(rfq_std_dev)
print(f"total RFQs = {len(time_passed_rfqs)}")

rfq_dict = {
               'min': rfq_min,
               'max': rfq_max,
               'median' : rfq_median,
               'average': rfq_mean,    
               'standard_deviation' : rfq_std_dev,
               '# of txs' : rfq_count}

225.20
221
249
207
225.20
total RFQs = 5


In [14]:
# Analysis of BID Transaction 
# only passed BIDs 
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))


print(f" bid_mean =  {bid_mean} \n bid_median = {bid_median} \n bid_max = {bid_max} \n"
     f" bid_std_dev = {bid_std_dev} \n total_bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : bid_count
              }

 bid_mean =  124.75 
 bid_median = 89 
 bid_max = 622 
 bid_std_dev = 105.82 
 total_bids = 81


In [15]:
# Analysis of ACCEPT Transaction

accept_mean =  "{:.2f}".format(stat.mean(time_passed_accepts))
accept_median = stat.median(time_passed_accepts)
accept_max = max(time_passed_accepts)
accept_min = min(time_passed_accepts)
accept_std_dev = "{:.2f}".format(stat.stdev(time_passed_accepts))

print(accept_mean)
print(accept_median)
print(accept_max)
print(accept_min)
print(accept_std_dev)
print(f"total ACCEPTs = {len(time_passed_accepts)}")

accept_dict = {
               'min': accept_min,
               'max': accept_max,
               'median' : accept_median,
               'average': accept_mean,
               'standard_deviation' : accept_std_dev,
               '# of txs' : accept_count}

2302.00
1846
4270
890
1294.24
total ACCEPTs = 5


In [16]:
# Analysis of RETURN Transaction

return_mean = "{:.2f}".format(stat.mean(time_passed_returns))
return_median = stat.median(time_passed_returns)
return_max = max(time_passed_returns)
return_min = min(time_passed_returns)
return_std_dev =  "{:.2f}".format(stat.stdev(time_passed_returns))

print(return_mean)
print(return_median)
print(return_max)
print(return_min)
print(return_std_dev)
print(f"total RETURNs = {len(time_passed_returns)}")
return_dict = {
               'min': return_min,
               'max': return_max,
               'median' : return_median,
               'average': return_mean,
               'standard_deviation' : return_std_dev,
               '# of txs' : len(time_passed_returns)}

2224.25
1823
4270
672
1135.25
total RETURNs = 81


In [17]:
list_of_dict = [create_dict, rfq_dict, bid_dict, accept_dict, return_dict]
results = pd.DataFrame(list_of_dict, index = ['CREATE','RFQ', 'BID', 'ACCEPT', 'RETURN'])

In [18]:
results

,min,max,average,median,standard_deviation,# of txs
CREATE,67,145,85.80,85.5,17.52,115
RFQ,207,249,225.20,221.0,225.20,5
BID,82,622,124.75,89.0,105.82,90
ACCEPT,890,4270,2302.00,1846.0,1294.24,5
RETURN,672,4270,2224.25,1823.0,1135.25,81


In [17]:
successfull_rate = len(time_passed_accepts) / accept_count * 100
successfull_rate

100.0

In [18]:
successfull_rate = len(time_passed_bids) / bid_count * 100
successfull_rate

99.8

In [19]:
successfull_rate = len(time_passed_creates) / create_count * 100
successfull_rate

99.79959919839679

In [22]:
successfull_rate = len(time_passed_returns) / return_count_1 * 100
successfull_rate

100.0

In [23]:
successfull_rate = len(time_passed_rfqs) / rfq_count * 100
successfull_rate

100.0

In [24]:
successfull_rate_accept = len(time_passed_accepts) / accept_count * 100
successfull_rate_bid = len(time_passed_bids) / bid_count * 100
successfull_rate_create = len(time_passed_creates) / create_count * 100
successfull_rate_return = len(time_passed_returns) / return_count_1 * 100
successfull_rate_rfq = len(time_passed_rfqs) / rfq_count * 100

print(f"rfq rate: {successfull_rate_rfq} %")
print(f"create rate: {successfull_rate_create} %")
print(f"bid rate: {successfull_rate_bid} %")
print(f"accept rate: {successfull_rate_accept} %")
print(f"return rate: {successfull_rate_return} %")

rfq rate: 100.0 %
create rate: 99.79959919839679 %
bid rate: 99.8 %
accept rate: 100.0 %
return rate: 100.0 %


overall transactions' size: 1,715,238 bytes
total time:  758.6
bandwidth: 441.8 Bps 